In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.emptyClustersChecker import *
from TPCQCVis.src.utility import *
from copy import copy, deepcopy
ROOT.gErrorIgnoreLevel = ROOT.kError

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
from ROOT import TFile, TF1, TCanvas, gStyle, TLatex

# Function to find peaks
def find_peaks_in_range(hist, x_min, x_max):
    peaks = []
    for bin in range(2, hist.GetNbinsX()):  # Ignore the first/last bins (under/overflow)
        left = hist.GetBinContent(bin - 1)
        center = hist.GetBinContent(bin)
        right = hist.GetBinContent(bin + 1)
        x_position = hist.GetBinCenter(bin)

        # Check if the bin is a peak AND within the specified range
        if center > left and center > right and x_min <= x_position <= x_max:
            peaks.append((center, x_position))

    # Sort peaks by height (descending)
    peaks.sort(reverse=True, key=lambda x: x[0])
    return peaks

# Define the double Gaussian with exponential background model
def double_gauss_with_exp(x, params):
    """
    Double Gaussian with exponential background.

    Parameters:
        x (list): The x-value (independent variable).
        params (list): The parameters for the fit function.
            params[0] = amplitude of first Gaussian
            params[1] = mean of first Gaussian
            params[2] = sigma of first Gaussian
            params[3] = amplitude of second Gaussian
            params[4] = mean of second Gaussian
            params[5] = sigma of second Gaussian
            params[6] = background amplitude (A)
            params[7] = background decay rate (k)

    Returns:
        float: Function value at x.
    """
    # First Gaussian
    G1 = params[0] * np.exp(-0.5 * ((x[0] - params[1]) / params[2])**2)

    # Second Gaussian
    G2 = params[3] * np.exp(-0.5 * ((x[0] - params[4]) / params[5])**2)

    # Exponential background
    B = params[6] * np.exp(-params[7] * x[0])

    # Background fit applied to specific range
    applyPieceWiseCondition = True
    if applyPieceWiseCondition:
        # Piecewise conditions
        if x[0] < 45:
            return G1 + G2
        elif x[0] >= 45:
            return G1 + G2 + B
    else:
        return G1 + G2 + B

def double_gauss_with_gauss_tail():
    """
    Double Gaussian with right tail background from far gaussian.

    Parameters:
        x (list): The x-value (independent variable).
        params (list): The parameters for the fit function.
            params[0] = amplitude of first Gaussian
            params[1] = mean of first Gaussian
            params[2] = sigma of first Gaussian
            params[3] = amplitude of second Gaussian
            params[4] = mean of second Gaussian
            params[5] = sigma of second Gaussian
            params[6] = amplitude of third Gaussian
            params[7] = mean of third Gaussian
            params[8] = sigma of third Gaussian

    Returns:
        float: Function value at x.
    """
    # First Gaussian
    G1 = params[0] * np.exp(-0.5 * ((x[0] - params[1]) / params[2])**2)

    # Second Gaussian
    G2 = params[3] * np.exp(-0.5 * ((x[0] - params[4]) / params[5])**2)

    # Exponential background
    G3 = params[6] * np.exp(-0.5 * ((x[0] - params[7]) / params[8])**2)

    # Background fit applied to specific range
    applyPieceWiseCondition = True
    if applyPieceWiseCondition:
        # Piecewise conditions
        if x[0] < 45:
            return G1 + G2
        elif x[0] >= 45:
            return G1 + G2 + G3
    else:
        return G1 + G2 + B
# Separation power function
def separationPower(QC_file, drawFit=True, getProjectionP=True, momentum_min = 0.35, momentum_max = 0.45, fit_type="simple"):
    """
    Calculates the separation power for particle identification using the `hdEdxTotMIP_TPC` histogram.
    
    This function fits a gaussian model (optionally with an exponential background) to the dE/dx distribution 
    and calculates the separation power between the two peaks (typically pions and electrons). 
    Two models are available: (i) simple gaussian for each peak (2 total gaussians), (ii) simple gaussian for 
    each peak with exponential model for background on right range.
    It supports both direct 1D histograms or projections from 2D dE/dx vs momentum histograms.
    
    Parameters:
        QC_file (str): Path to the ROOT `_QC.root` file containing the histogram data.
        drawFit (bool, optional): If True, fits the histogram and displays the fit on a canvas. Default is True.
        getProjectionP (bool, optional): If True, extracts a dE/dx histogram by projecting a specific momentum range
                                         from a 2D histogram. Default is True.
        momentum_min (float, optional): Lower bound of the momentum range for projection. Used if `getProjectionP=True`.
                                        Default is 0.28.
        momentum_max (float, optional): Upper bound of the momentum range for projection. Used if `getProjectionP=True`.
                                        Default is 0.4.
        fit_type (str, optional): The type of fit model to use. Options are:
            - "simple": Double-Gaussian fit.
            - "simple_with_exp": Double-Gaussian with an exponential background fit. Default is "simple".
    
    Returns:
        float: The calculated separation power (|Mean1 - Mean2| / sqrt(0.5 * (Sigma1^2 + Sigma2^2))).
        TCanvas: A ROOT TCanvas object displaying the histogram and fit, if `drawFit=True`.
    """
    # ----- Access histogram
    if getProjectionP: # Get projection on appropriate momentum range
        hist2D = QC_file.Get("PIDQC/hdEdxTotVsP_TPC") # "PIDQC/hdEdxMaxVsP_TPC", "PIDQC/hdEdxTotVsP_TPC"
        if not hist2D:
            print("Histogram 'hdEdxTotVsP_TPC' not found in the file")
            return
        else:
            # Project the 2D histogram onto the dEdx axis for the given momentum range
            x_bin_min = hist2D.GetXaxis().FindBin(momentum_min)
            x_bin_max = hist2D.GetXaxis().FindBin(momentum_max)
            hist = hist2D.ProjectionY("hdEdxTotVsP_TPC", x_bin_min, x_bin_max)
            if not hist:
                print("Projection failed. Check the momentum range or 2D histogram")
                return
            # Restrict the dEdx range for the projected histogram
            hist.GetXaxis().SetRangeUser(15, 150)
    else: # Get the histogram directly on given fixed momentum range (which range is that?)
        hist = QC_file.Get("PIDQC/hdEdxTotMIP_TPC")
        if not hist:
            print("Histogram 'hdEdxTotMIP_TPC' not found in the file.")
            return

    # ----- Perform double gaussian fit
    fit_func = None # initialize the fit_func object outside the conditional clause
    if fit_type == "double_gaussian":
        x_min_displayed = hist.GetXaxis().GetBinLowEdge(hist.GetXaxis().GetFirst())
        x_max_displayed = hist.GetXaxis().GetBinUpEdge(hist.GetXaxis().GetLast())
        print("x_min_displayed: ", x_min_displayed)
        print("x_max_displayed: ", x_max_displayed)
        print("hist.GetXaxis().GetXmin(): ", hist.GetXaxis().GetXmin())
        print("hist.GetXaxis().GetXmax(): ", hist.GetXaxis().GetXmax())
        # define the simple Gaussians function
        fit_func = TF1("double_gaussian",
                       "[0]*exp(-0.5*((x-[1])/[2])**2) + [3]*exp(-0.5*((x-[4])/[5])**2)",
                       x_min_displayed,
                       x_max_displayed)

        # Set initial parameter guesses
        peaks = find_peaks_in_range(hist, 20, 150) # tuples of (height, x_position)
        print("Peaks: ", peaks)
        # First peak (pions)
        peak1_mean = peaks[0][1]  # Mean for the first Gaussian
        peak1_amp = peaks[0][0]  # Amplitude for the first Gaussian
        peak1_sigma = hist.GetBinWidth(1) * 1  # Assume a width spanning ~5 bins
        # Second peak (electrons): assume it is to the right of the first
        peak2_mean = peaks[1][1]  # Mean for the second Gaussian
        peak2_amp = peaks[1][0]  # Amplitude for the second Gaussian
        peak2_sigma = peak1_sigma * 5.5  # Second peak is wider. Default = peak1_sigma * 0.5
        ## Set parameter guesses: [Amp1, Mean1, Sigma1, Amp2, Mean2, Sigma2]
        fit_func.SetParameters(peak1_amp, peak1_mean, peak1_sigma, peak2_amp, peak2_mean, peak2_sigma)
        fit_func.SetParName(0, "Amp1 (Pions)")  # Amplitude of the first Gaussian (Pions)
        fit_func.SetParName(1, "Mean1 (Pions)")  # Mean of the first Gaussian
        fit_func.SetParName(2, "Sigma1 (Pions)")  # Width (sigma) of the first Gaussian
        fit_func.SetParName(3, "Amp2 (Electrons)")  # Amplitude of the second Gaussian (Electrons)
        fit_func.SetParName(4, "Mean2 (Electrons)")  # Mean of the second Gaussian
        fit_func.SetParName(5, "Sigma2 (Electrons)")  # Width (sigma) of the second Gaussian
        # Applying constrains to the parameters
        fit_func.SetParLimits(4, peak2_mean - 5, peak2_mean + 5)  # Mean of second peak
        fit_func.SetParLimits(5, peak1_sigma * 5.0, peak1_sigma * 20.0)  # Width of second peak
        #fit_func.SetParLimits(3, 0, 1e30) # Amplitude of second peak positive
        #fit_func.SetParLimits(4, 75, 95) # Mean of second peak positive

        # Fit the histogram
        fit_result = hist.Fit(fit_func, "QS") # "Q" = quiet mode (no verbose), "S" = draw and print fitted parameters
        if not fit_result:
            print("WARNING: fit failed. Check the initial parameter guesses or data quality.")
            return
        
        # Extract fit parameters
        amp1, mean1, sigma1 = fit_func.GetParameter(0), fit_func.GetParameter(1), fit_func.GetParameter(2)
        amp2, mean2, sigma2 = fit_func.GetParameter(3), fit_func.GetParameter(4), fit_func.GetParameter(5)
        print("Gaussian 1 fit parameters: ", amp1, mean1, sigma1)
        print("Gaussian 2 fit parameters: ", amp2, mean2, sigma2)

        # Calculate the separation power
        separation_power = (mean2 - mean1) / ((sigma2 + sigma1) / 2)
    elif fit_type == "triple_gaussian":
        # define the fit function: double Gaussian with exponential after the first peak
        x_min_displayed = hist.GetXaxis().GetBinLowEdge(hist.GetXaxis().GetFirst())
        x_max_displayed = hist.GetXaxis().GetBinUpEdge(hist.GetXaxis().GetLast())
        fit_func = TF1("triple_gaussian", 
                       "[0]*exp(-0.5*((x-[1])/[2])**2) + [3]*exp(-0.5*((x-[4])/[5])**2) + [6]*exp(-0.5*((x-[7])/[8])**2)", 
                       x_min_displayed, 
                       x_max_displayed)

        # Initial guesses for parameters
        fit_func.SetParameters(1000, 40, 5, 500, 100, 7, 100, 20, 5)  # Adjust guesses based on your data

        # Set initial parameter guesses
        peaks = find_peaks_in_range(hist, 20, 150) # tuples of (height, x_position)
        print("Peaks: ", peaks)
        # First peak (pions)
        peak1_mean = peaks[0][1]  # Mean for the first Gaussian
        peak1_amp = peaks[0][0]  # Amplitude for the first Gaussian
        peak1_sigma = hist.GetBinWidth(1) * 1  # Assume a width spanning ~5 bins
        # Second peak (electrons): assume it is to the right of the first
        peak2_mean = peaks[1][1]  # Mean for the second Gaussian
        peak2_amp = peaks[1][0]  # Amplitude for the second Gaussian
        peak2_sigma = peak1_sigma * 5.5  # Second peak is wider. Default = peak1_sigma * 5.5
        # Second peak (kaons): assume it is to the right of the first
        peak3_mean = peaks[2][1]  # Mean for the third Gaussian
        peak3_amp = peaks[2][0]  # Amplitude for the third Gaussian
        peak3_sigma = peak1_sigma * 5.5  # Second peak is wider. Default = peak1_sigma * 0.5
        ## Apply parameter guesses: [Amp1, Mean1, Sigma1, Amp2, Mean2, Sigma2, Amp3, Mean3, Sigma3]
        # Exponential background
        back_amp = peak1_amp * 0.5 # exponential amplitude
        back_decay = 0.01 # exponential decay rate
        fit_func.SetParameters(peak1_amp, peak1_mean, peak1_sigma, peak2_amp, peak2_mean, peak2_sigma, peak3_amp, peak3_mean, peak3_sigma)
        fit_func.SetParName(0, "Amp1 (Pions)")  # Amplitude of the first Gaussian (Pions)
        fit_func.SetParName(1, "Mean1 (Pions)")  # Mean of the first Gaussian
        fit_func.SetParName(2, "Sigma1 (Pions)")  # Width (sigma) of the first Gaussian
        fit_func.SetParName(3, "Amp2 (Electrons)")  # Amplitude of the second Gaussian (Electrons)
        fit_func.SetParName(4, "Mean2 (Electrons)")  # Mean of the second Gaussian
        fit_func.SetParName(5, "Sigma2 (Electrons)")  # Width (sigma) of the second Gaussian
        fit_func.SetParName(6, "Amp3 (Kaons)")  # Amplitude of the third Gaussian (Kaons)
        fit_func.SetParName(7, "Mean3 (Kaons)")  # Mean of the third Gaussian
        fit_func.SetParName(8, "Sigma3 (Kaons)")  # Width (sigma) of the third Gaussian

        # Applying constrains to the parameters
        #fit_func.SetParLimits(3, 0, peak1_amp * 1.0)  # Amplitude of second peak
        print("peak3 limits: ", peak3_mean - 5, peak3_mean + 5)
        fit_func.SetParLimits(4, peak2_mean - 5, peak2_mean + 5)  # Mean of second peak
        fit_func.SetParLimits(5, peak1_sigma * 5.0, peak1_sigma * 20.0)  # Width of second peak
        fit_func.SetParLimits(7, peak3_mean - 5, peak3_mean + 5)  # Mean of third peak
        fit_func.SetParLimits(8, peak1_sigma * 5.0, peak1_sigma * 100.0)  # Width of third peak

        # Fit the histogram
        fit_result = hist.Fit(fit_func, "QS") # "Q" = quiet mode (no verbose), "S" = draw and print fitted parameters
        if not fit_result:
            print("WARNING: fit failed. Check the initial parameter guesses or data quality.")
            return
        
        # Extract fit parameters
        amp1, mean1, sigma1 = fit_func.GetParameter(0), fit_func.GetParameter(1), fit_func.GetParameter(2)
        amp2, mean2, sigma2 = fit_func.GetParameter(3), fit_func.GetParameter(4), fit_func.GetParameter(5)
        amp3, mean3, sigma3 = fit_func.GetParameter(6), fit_func.GetParameter(7), fit_func.GetParameter(8)
        print("Gaussian 1 fit parameters: ", amp1, mean1, sigma1)
        print("Gaussian 2 fit parameters: ", amp2, mean2, sigma2)
        print("Gaussian 3 fit parameters: ", amp3, mean3, sigma3)

        # Calculate the separation power
        separation_power = (mean2 - mean1) / ((sigma2 + sigma1) / 2)
    elif fit_type == "double_gaussian_with_exp":
        # define the fit function: double Gaussian with exponential after the first peak
        x_min_displayed = hist.GetXaxis().GetBinLowEdge(hist.GetXaxis().GetFirst())
        x_max_displayed = hist.GetXaxis().GetBinUpEdge(hist.GetXaxis().GetLast())
        fit_func = TF1("double_gauss_with_exp", 
                       double_gauss_with_exp, 
                       x_min_displayed, 
                       x_max_displayed, 
                       8)

        # Initial guesses for parameters
        fit_func.SetParameters(1000, 40, 5, 500, 100, 7, 50, 0.01)  # Adjust guesses based on your data

        # Set initial parameter guesses
        peaks = find_peaks_in_range(hist, 20, 150) # tuples of (height, x_position)
        print("Peaks: ", peaks)
        # First peak (pions)
        peak1_mean = peaks[0][1]  # Mean for the first Gaussian
        peak1_amp = peaks[0][0]  # Amplitude for the first Gaussian
        peak1_sigma = hist.GetBinWidth(1) * 1  # Assume a width spanning ~5 bins
        # Second peak (electrons): assume it is to the right of the first
        peak2_mean = peaks[1][1]  # Mean for the second Gaussian
        peak2_amp = peaks[1][0]  # Amplitude for the second Gaussian
        peak2_sigma = peak1_sigma * 1.5  # Second peak is thinner?
        ## Apply parameter guesses: [Amp1, Mean1, Sigma1, Amp2, Mean2, Sigma2]
        # Exponential background
        back_amp = peak1_amp * 0.5 # exponential amplitude
        back_decay = 0.01 # exponential decay rate
        fit_func.SetParameters(peak1_amp, peak1_mean, peak1_sigma, peak2_amp, peak2_mean, peak2_sigma, back_amp, back_decay)
        fit_func.SetParName(0, "Amp1 (Pions)")  # Amplitude of the first Gaussian (Pions)
        fit_func.SetParName(1, "Mean1 (Pions)")  # Mean of the first Gaussian
        fit_func.SetParName(2, "Sigma1 (Pions)")  # Width (sigma) of the first Gaussian
        fit_func.SetParName(3, "Amp2 (Electrons)")  # Amplitude of the second Gaussian (Electrons)
        fit_func.SetParName(4, "Mean2 (Electrons)")  # Mean of the second Gaussian
        fit_func.SetParName(5, "Sigma2 (Electrons)")  # Width (sigma) of the second Gaussian
        fit_func.SetParName(6, "Amp (Background)")  # Width (sigma) of the second Gaussian
        fit_func.SetParName(7, "Decay rate (Background)")  # Width (sigma) of the second Gaussian

        # Applying constrains to the parameters
        #fit_func.SetParLimits(3, 0, peak1_amp * 1.0)  # Amplitude of second peak
        print("peak2 limits: ", peak2_mean, peak2_mean + 5)
        fit_func.SetParLimits(4, peak2_mean, peak2_mean + 5)  # Mean of second peak
        fit_func.SetParLimits(5, peak1_sigma * 1.0, peak1_sigma * 10.0)  # Width of second peak
        #fit_func.SetParLimits(6, peak1_amp * 0.3, peak1_amp * 10.0)  # Amplitude of background exponential

        # Fit the histogram
        fit_result = hist.Fit(fit_func, "QS") # "Q" = quiet mode (no verbose), "S" = draw and print fitted parameters
        if not fit_result:
            print("WARNING: fit failed. Check the initial parameter guesses or data quality.")
            return
        
        # Extract fit parameters
        amp1, mean1, sigma1 = fit_func.GetParameter(0), fit_func.GetParameter(1), fit_func.GetParameter(2)
        amp2, mean2, sigma2 = fit_func.GetParameter(3), fit_func.GetParameter(4), fit_func.GetParameter(5)
        print("Gaussian 1 fit parameters: ", amp1, mean1, sigma1)
        print("Gaussian 2 fit parameters: ", amp2, mean2, sigma2)

        # Calculate the separation power
        separation_power = (mean2 - mean1) / ((sigma2 + sigma1) / 2)
    elif fit_type == "double_gaussian_and_tail":
        # define the fit function: double Gaussian with exponential after the first peak
        x_min_displayed = hist.GetXaxis().GetBinLowEdge(hist.GetXaxis().GetFirst())
        x_max_displayed = hist.GetXaxis().GetBinUpEdge(hist.GetXaxis().GetLast())
        fit_func = TF1("double_gauss_with_exp", 
                       double_gauss_with_exp, 
                       x_min_displayed, 
                       x_max_displayed, 
                       8)

        # Initial guesses for parameters
        fit_func.SetParameters(1000, 40, 5, 500, 100, 7, 50, 0.01)  # Adjust guesses based on your data

        # Set initial parameter guesses
        peaks = find_peaks_in_range(hist, 20, 150) # tuples of (height, x_position)
        print("Peaks: ", peaks)
        # First peak (pions)
        peak1_mean = peaks[0][1]  # Mean for the first Gaussian
        peak1_amp = peaks[0][0]  # Amplitude for the first Gaussian
        peak1_sigma = hist.GetBinWidth(1) * 1  # Assume a width spanning ~5 bins
        # Second peak (electrons): assume it is to the right of the first
        peak2_mean = peaks[1][1]  # Mean for the second Gaussian
        peak2_amp = peaks[1][0]  # Amplitude for the second Gaussian
        peak2_sigma = peak1_sigma * 1.5  # Second peak is thinner?
        # Third peak (kaons): assume it is to the very far left of the first
        peak3_mean = peak1_mean - 100  # Mean for the second Gaussian
        peak3_amp = peak1_amp * 10  # Amplitude for the second Gaussian
        peak3_sigma = peak1_sigma * 10  # Second peak is thinner?
        ## Apply parameter guesses: [Amp1, Mean1, Sigma1, Amp2, Mean2, Sigma2, Amp3, Mean3, Sigma3]
        fit_func.SetParameters(peak1_amp, peak1_mean, peak1_sigma, peak2_amp, peak2_mean, peak2_sigma, peak3_amp, peak3_mean, peak3_sigma)
        fit_func.SetParName(0, "Amp1 (Pions)")  # Amplitude of the first Gaussian (Pions)
        fit_func.SetParName(1, "Mean1 (Pions)")  # Mean of the first Gaussian
        fit_func.SetParName(2, "Sigma1 (Pions)")  # Width (sigma) of the first Gaussian
        fit_func.SetParName(3, "Amp2 (Electrons)")  # Amplitude of the second Gaussian (Electrons)
        fit_func.SetParName(4, "Mean2 (Electrons)")  # Mean of the second Gaussian
        fit_func.SetParName(5, "Sigma2 (Electrons)")  # Width (sigma) of the second Gaussian
        fit_func.SetParName(6, "Amp3 (Kaons)")  # Amplitude of the first Gaussian (Kaons)
        fit_func.SetParName(7, "Mean3 (Kaons)")  # Mean of the first Gaussian
        fit_func.SetParName(8, "Sigma3 (Kaons)")  # Width (sigma) of the first Gaussian

        # Applying constrains to the parameters
        #fit_func.SetParLimits(3, 0, peak1_amp * 1.0)  # Amplitude of second peak
        print("peak2 limits: ", peak2_mean, peak2_mean + 5)
        fit_func.SetParLimits(4, peak2_mean, peak2_mean + 5)  # Mean of second peak
        fit_func.SetParLimits(5, peak1_sigma * 1.0, peak1_sigma * 10.0)  # Width of second peak
        #fit_func.SetParLimits(6, peak1_amp * 0.3, peak1_amp * 10.0)  # Amplitude of background exponential

        # Fit the histogram
        fit_result = hist.Fit(fit_func, "QS") # "Q" = quiet mode (no verbose), "S" = draw and print fitted parameters
        if not fit_result:
            print("WARNING: fit failed. Check the initial parameter guesses or data quality.")
            return
        
        # Extract fit parameters
        amp1, mean1, sigma1 = fit_func.GetParameter(0), fit_func.GetParameter(1), fit_func.GetParameter(2)
        amp2, mean2, sigma2 = fit_func.GetParameter(3), fit_func.GetParameter(4), fit_func.GetParameter(5)
        amp3, mean3, sigma3 = fit_func.GetParameter(6), fit_func.GetParameter(7), fit_func.GetParameter(8)
        print("Gaussian 1 fit parameters: ", amp1, mean1, sigma1)
        print("Gaussian 2 fit parameters: ", amp2, mean2, sigma2)
        print("Gaussian 3 fit parameters: ", amp3, mean3, sigma3)

        # Calculate the separation power
        separation_power = (mean2 - mean1) / ((sigma2 + sigma1) / 2)

    # ----- Draw distribution
    canvas = TCanvas("c1", "hdEdxTotMIP_TPC_"+fit_type, 800, 600)
    canvas.cd()
    hist.SetStats(0)
    hist.Draw()  # Draw histogram with error bars
    
    # Optionally also draw the fit with the distribution
    if drawFit:
        if getProjectionP:
            hist.SetTitle("hdEdxTotVsP_TPC projection and separation power fit")
        else:
            hist.SetTitle("hdEdxTotMIP_TPC and separation power fit")
        gStyle.SetOptFit(0)  # Show fit stats box = 1
        fit_result.Draw("SAME")  # Overlay fit on histogram

        # Add separation power text using TLatex
        latex = TLatex()
        latex.SetTextSize(0.03)
        latex.DrawLatexNDC(0.6, 0.85, "Separation Power (#pi/e^{{-}}): {:.5f}".format(separation_power))
        chi2_reduced = fit_result.Chi2() / fit_result.Ndf()
        print("#Chi^{2} = ", fit_result.Chi2())
        print("Number of Degrees of Freedom = ", fit_result.Ndf())
        latex.DrawLatexNDC(0.6, 0.80, f"#Chi^{{2}} / NDF = {chi2_reduced:.2f}")
        if getProjectionP:
            latex.DrawLatexNDC(0.6, 0.75, f"{momentum_min:.2f} GeV/c < p < {momentum_max:.2f} GeV/c")

        # Display particle ID info por each peak
        if fit_type == "double_gaussian":
            # Define positions (use your actual fit peak positions)
            pi_x, pi_y = mean1, amp1
            e_x, e_y = mean2, amp2
            # Create text labels
            text = TLatex()
            text.SetTextSize(0.02)  # Adjust text size
            text.DrawLatex(pi_x, pi_y * 1.01, "Pions")
            text.DrawLatex(e_x, e_y * 1.15, "Electrons")
            #latex.DrawLatexNDC(pi_x, pi_y * 1.05, f"Pion peak: {mean1:.2f} #pm {sigma1:.2f}")
            #latex.DrawLatexNDC(e_x, e_y * 1.05, f"Electron peak: {mean2:.2f} #pm {sigma2:.2f}")
        elif fit_type == "triple_gaussian":
            # Define positions (use your actual fit peak positions)
            pi_x, pi_y = mean1, amp1
            e_x, e_y = mean2, amp2
            k_x, k_y = mean3, amp3
            # Create text labels
            text = TLatex()
            text.SetTextSize(0.02)  # Adjust text size
            text.DrawLatex(pi_x, pi_y * 1.01, "Pions")
            text.DrawLatex(e_x, e_y * 1.15, "Electrons")
            text.DrawLatex(k_x, k_y * 1.15, "Kaons")
        elif fit_type == "double_gaussian_with_exp":
            # Define positions (use your actual fit peak positions)
            pi_x, pi_y = mean1, amp1
            e_x, e_y = mean2, amp2
            # Create text labels
            text = TLatex()
            text.SetTextSize(0.02)  # Adjust text size
            text.DrawLatex(pi_x, pi_y * 1.01, "Pions")
            text.DrawLatex(e_x, e_y * 1.15, "Electrons")
        canvas.Update()

    return separation_power, canvas

In [ ]:
# Notebook variables
periodName = "myPeriod"
passName = "myPass"
runNumber = 123456
path = "myPath"
num_gaussians = int("numGaussians") # number of gaussians in separation power fit (between pions and electrons)

# periodName = "LHC23zzk"
# passName = "apass4"
# runNumber = 544451
# path = "/home/ferrandi/alice/TPCQCVis/TPCQCVis/DownloadedData/2023"

#periodName = "LHC23f"
#passName = "apass4"
#runNumber = "535069"
#path = "$TPCQCVIS_DATA/2023/"

# Read QC root file
rootDataFile=[]
rootDataFile.append(ROOT.TFile.Open(path+"/"+periodName+"/"+passName+"/"+str(runNumber)+"_QC.root","READ"))

# TPC Async QC Report - myPeriod - myPass
**[RCT Table](https://ali-bookkeeping.cern.ch/?page=lhc-period-overview)**

In [ ]:
import datetime
now = datetime.datetime.now()
display(Markdown("Report automatically created on **"+now.strftime("%d.%m.%Y @ %H:%M:%S")+"**"))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Run-Parameters" data-toc-modified-id="Run-Parameters-1">Run Parameters</a></span></li></ul><ul class="toc-item"><li><span><a href="#Per-Pad-(Clusters)" data-toc-modified-id="Per-Pad-(Clusters)-1">Per Pad (Clusters)</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#nClusters" data-toc-modified-id="nClusters-1.0.1">nClusters</a></span></li><li><span><a href="#Q_Max" data-toc-modified-id="Q_Max-1.0.2">Q_Max</a></span></li><li><span><a href="#Q_Tot" data-toc-modified-id="Q_Tot-1.0.3">Q_Tot</a></span></li><li><span><a href="#Time_Bin" data-toc-modified-id="Time_Bin-1.0.4">Time_Bin</a></span></li><li><span><a href="#Sigma_Pad" data-toc-modified-id="Sigma_Pad-1.0.5">Sigma_Pad</a></span></li><li><span><a href="#Sigma_Time" data-toc-modified-id="Sigma_Time-1.0.6">Sigma_Time</a></span></li></ul></li></ul></li><li><span><a href="#Tracks" data-toc-modified-id="Tracks-2">Tracks</a></span><ul class="toc-item"><li><span><a href="#Geometrical-distributions" data-toc-modified-id="Geometrical-distributions-2.1">Geometrical distributions</a></span><ul class="toc-item"><li><span><a href="#Phi" data-toc-modified-id="Phi-2.1.1">Phi</a></span></li><li><span><a href="#Eta" data-toc-modified-id="Eta-2.1.2">Eta</a></span></li><li><span><a href="#Eta-vs-phi" data-toc-modified-id="Eta-vs-phi-2.1.3">Eta vs phi</a></span></li></ul></li><li><span><a href="#Track-properties" data-toc-modified-id="Track-properties-2.2">Track properties</a></span><ul class="toc-item"><li><span><a href="#Tracks-Clusters" data-toc-modified-id="Tracks-Clusters-2.2.1">Tracks Clusters</a></span></li><li><span><a href="#nClusters-vs-eta" data-toc-modified-id="nClusters-vs-eta-2.2.2">nClusters vs eta</a></span></li><li><span><a href="#Moving-window-nClusters_vs_eta" data-toc-modified-id="Moving-window-nClusters_vs_eta-2.2.3">Moving window nClusters_vs_eta</a></span></li><li><span><a href="#nClusters-vs-pT" data-toc-modified-id="nClusters-vs-pT-2.2.4">nClusters vs pT</a></span></li><li><span><a href="#nClusters-vs-phi-(A-Side)" data-toc-modified-id="nClusters-vs-phi-(A-Side)-2.2.5">nClusters vs phi (A-Side)</a></span></li><li><span><a href="#nClusters-vs-phi-(C-Side)" data-toc-modified-id="nClusters-vs-phi-(C-Side)-2.2.6">nClusters vs phi (C-Side)</a></span></li><li><span><a href="#pT" data-toc-modified-id="pT-2.2.7">pT</a></span></li><li><span><a href="#Q/pT" data-toc-modified-id="Q/pT-2.2.8">Q/pT</a></span></li></ul></li><li><span><a href="#DCA-of-tracks" data-toc-modified-id="DCA-of-tracks-2.3">DCA of tracks</a></span><ul class="toc-item"><li><span><a href="#DCAr-vs-phi" data-toc-modified-id="DCAr-vs-phi-2.3.1">DCAr vs phi</a></span></li><li><span><a href="#DCAr-vs-eta" data-toc-modified-id="DCAr-vs-eta-2.3.2">DCAr vs eta</a></span></li><li><span><a href="#DCAr-vs-nClusters" data-toc-modified-id="DCAr-vs-nClusters-2.3.3">DCAr vs nClusters</a></span></li><li><span><a href="#DCAr-vs-pT" data-toc-modified-id="DCAr-vs-pT-2.3.4">DCAr vs pT</a></span></li></ul></li><li><span><a href="#DCA-high-pT-tracks" data-toc-modified-id="DCA-high-pT-tracks-2.4">DCA high pT tracks</a></span><ul class="toc-item"><li><span><a href="#DCAr-vs-phi-with-pT-cut-(>1-GeV)" data-toc-modified-id="DCAr-vs-phi-with-pT-cut-(>1-GeV)-2.4.1">DCAr vs phi with pT cut (&gt;1 GeV)</a></span></li><li><span><a href="#DCAr-vs-eta-with-pT-cut-(>1-GeV)" data-toc-modified-id="DCAr-vs-eta-with-pT-cut-(>1-GeV)-2.4.2">DCAr vs eta with pT cut (&gt;1 GeV)</a></span></li><li><span><a href="#DCAr-vs-nClusters-with-pT-cut-(>1-GeV)" data-toc-modified-id="DCAr-vs-nClusters-with-pT-cut-(>1-GeV)-2.4.3">DCAr vs nClusters with pT cut (&gt;1 GeV)</a></span></li></ul></li></ul></li><li><span><a href="#Energy-Loss" data-toc-modified-id="Energy-Loss-3">Energy Loss</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#dEdxTot-for-MIPs" data-toc-modified-id="dEdxTot-for-MIPs-3.0.1">dEdxTot for MIPs</a></span></li><li><span><a href="#Moving-window-MIP-dEdxTot" data-toc-modified-id="Moving-window-MIP-dEdxTot-3.0.2">Moving window MIP dEdxTot</a></span></li><li><span><a href="#dEdxTotMip-vs-sector-(phi)" data-toc-modified-id="dEdxTotMip-vs-sector-(phi)-3.0.3">dEdxTotMip vs sector (phi)</a></span></li><li><span><a href="#dEdxTotMip-vs-nClusters" data-toc-modified-id="dEdxTotMip-vs-nClusters-3.0.4">dEdxTotMip vs nClusters</a></span></li><li><span><a href="#dEdxTotMip-vs-tan(lambda)" data-toc-modified-id="dEdxTotMip-vs-tan(lambda)-3.0.5">dEdxTotMip vs tan(lambda)</a></span></li><li><span><a href="#dEdxTotMip-vs-sin(phi)" data-toc-modified-id="dEdxTotMip-vs-sin(phi)-3.0.6">dEdxTotMip vs sin(phi)</a></span></li><li><span><a href="#dEdxTot-vs-p" data-toc-modified-id="dEdxTot-vs-p-3.0.7">dEdxTot vs p</a></span></li><li><span><a href="#dEdxTot-vs-p-for-positive-and-negative-charges" data-toc-modified-id="dEdxTot-vs-p-for-positive-and-negative-charges-3.0.8">dEdxTot vs p for positive and negative charges</a></span></li><li><span><a href="#dEdxMax-for-MIPs" data-toc-modified-id="dEdxMax-for-MIPs-3.0.9">dEdxMax for MIPs</a></span></li><li><span><a href="#dEdxMaxMip-vs-sector-(phi)" data-toc-modified-id="dEdxMaxMip-vs-sector-(phi)-3.0.10">dEdxMaxMip vs sector (phi)</a></span></li><li><span><a href="#dEdxMaxMip-vs-nClusters" data-toc-modified-id="dEdxMaxMip-vs-nClusters-3.0.11">dEdxMaxMip vs nClusters</a></span></li><li><span><a href="#dEdxMaxMip-vs-tan(lambda)" data-toc-modified-id="dEdxMaxMip-vs-tan(lambda)-3.0.12">dEdxMaxMip vs tan(lambda)</a></span></li><li><span><a href="#dEdxMaxMip-vs-sin(phi)" data-toc-modified-id="dEdxMaxMip-vs-sin(phi)-3.0.13">dEdxMaxMip vs sin(phi)</a></span></li><li><span><a href="#dEdxMax-vs-p" data-toc-modified-id="dEdxMax-vs-p-3.0.14">dEdxMax vs p</a></span></li></ul></li></ul></li></ul></div>

## Run Parameters


In [ ]:
%jsroot on

RunParamTree = rootDataFile[0].Get("RunParameters")
if RunParamTree:
    '''
    IntRate = RunParamTree.GetBranch("IR")
    Duration = RunParamTree.GetBranch("Duration")
    BField = RunParamTree.GetBranch("BField")
    '''
    RunParamTree.GetEntry(0)
    IntRateAvg = getattr(RunParamTree, "IRavg")
    Duration = getattr(RunParamTree, "Duration")
    BField = getattr(RunParamTree, "BField")
    RunParamTable = f"""
    <table style="border: 1px solid black; width: 50%;">
    <tr>
        <th style="background-color: #f2f2f2; text-align: center;">Run Number</th>
        <th style="background-color: #f2f2f2; text-align: center;">Interaction Rate (Hz)</th>
        <th style="background-color: #f2f2f2; text-align: center;">Duration (s)</th>
        <th style="background-color: #f2f2f2; text-align: center;">Magnetic Field (Amperes)</th>
    </tr>
    """ # Table header
    RunParamTable += f"""
            <tr style="text-align: center;">
                <td style="text-align: center;"><code>{runNumber}</code></td>
                <td style="text-align: center;"><code>{IntRateAvg}</code></td>
                <td style="text-align: center;"><code>{Duration}</code></td>
                <td style="text-align: center;"><code>{BField}</code></td>
            </tr>
        """ # Table row
    display(HTML(RunParamTable))
else:
    display(HTML(f"No RunParameters tree found for run <code>{runNumber}</code>. To produce it, use runPlotter.py with <code>--add_run_param</code> option."))

## Per Pad (Clusters)

#### nClusters

In [ ]:
%jsroot on
objectName = "c_N_Clusters"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

#### Q_Max

In [ ]:
%jsroot on
objectName = "c_Q_Max"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

#### Q_Tot

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

#### Time_Bin

In [ ]:
%jsroot on
objectName = "c_Time_Bin"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

#### Sigma_Pad

In [ ]:
%jsroot on
objectName = "c_Sigma_Pad"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

#### Sigma_Time

In [ ]:
%jsroot on
objectName = "c_Sigma_Time"
if checkIfExists(rootDataFile,objectName):
    c = rootDataFile[0].ClusterQC.Get(objectName)
    outliers,centers,hulls =  detectOutlierRegions(c)
    c.Draw()

## Tracks

In [ ]:
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)

### Geometrical distributions

#### Phi

In [ ]:
%jsroot on
objectName="hPhiAside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="HIST",size=[1200,400])
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    canvas.SetCanvasSize(1000,400)
    pad1.SetGridy(1)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="HIST",size=[1200,400])
    #Sector boundatries
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,hist[0].GetMinimum(),(j*2*math.pi)/18,hist[0].GetMaximum())
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,hist[0].GetMaximum(),str(j))
    canvas.SetCanvasSize(1000,400)
    pad1.SetGridy(1)
    canvas.Draw()

#### Eta

In [ ]:
%jsroot on
objectName="hEta;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,xAxisRange = [-1.1,1.1],drawOption="HIST",size=[800,400])
    canvas.SetCanvasSize(700,400)
    pad1.SetGrid(1)
    canvas.Draw()

#### Eta vs phi

In [ ]:
%jsroot on
objectName="h2DEtaPhi;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-1,1],size=[1200,400])
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-1,(j*2*math.pi)/18,1)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,1.05,str(j))
    canvas.SetCanvasSize(1000,400)
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    #hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

### Track properties

#### Tracks Clusters

In [ ]:
%jsroot on
objectNames=["hNClustersAfterCuts","crossedRows","sharedClusters"]
if all([checkIfExists(rootDataFile,objectName) for objectName in objectNames]):
    bigCanvas = ROOT.TCanvas("","",1200,400)
    bigCanvas.Divide(3,1)
    [hist1,legend1,canvas1,pad1] = drawHistograms(objectNames[0],rootDataFile,xAxisRange = [50,160],drawOption="HIST")
    [hist2,legend2,canvas2,pad2] = drawHistograms(objectNames[1],rootDataFile,xAxisRange = [50,160],drawOption="HIST")
    [hist3,legend3,canvas3,pad3] = drawHistograms(objectNames[2],rootDataFile,xAxisRange = [0,160],drawOption="HIST")
    #canvas.SetCanvasSize(700,400)
    #hist.SetGrid(1)
    bigCanvas.cd(1)
    ROOT.gPad.SetGrid(1)
    hist1[0].Draw()
    bigCanvas.cd(2)
    ROOT.gPad.SetGrid(1)
    hist2[0].Draw()
    bigCanvas.cd(3)
    ROOT.gPad.SetGrid(1)
    hist3[0].Draw()
    ROOT.gPad.SetLogy()
    bigCanvas.Draw()

#### nClusters vs eta

In [ ]:
%jsroot on
objectName="h2DNClustersEta;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",size=[800,400])#,xAxisRange = [-1,1],yAxisRange = [60,180])
    canvas.SetCanvasSize(700,400)
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME LP")
    hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### Moving window nClusters_vs_eta

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
if checkIfExists(rootDataFile[0],objectName+"_mw"):
    folder = getHistogram(rootDataFile[0],objectName+"_mw")
    histTimestamps = list(set([item.GetName() for item in folder.GetListOfKeys()]))
    histTimestamps.sort()
    hists = [quantileProfile([folder.Get(time+";1")],quantileOrder=0.5)[0] for time in histTimestamps]
    canvas,leg = drawMovingWindowOverlay(hists,histTimestamps)
    leg.Draw()
    for hist in hists:
        hist.GetXaxis().SetRangeUser(-1.2,1.2)
        hist.GetYaxis().SetRangeUser(90,155)
    canvas.Draw()

#### nClusters vs pT

In [ ]:
%jsroot on
objectName="h2DNClustersPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logx",yAxisRange = [60,180],size=[800,400])
    canvas.SetCanvasSize(700,400)
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME LP")
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### nClusters vs phi (A-Side)

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",size=[1200,400])#,yAxisRange = [60,180])
    canvas.SetCanvasSize(1000,400)
    pad1.cd()
    
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,60,(j*2*math.pi)/18,180)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,175,str(j))
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME L P")
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### nClusters vs phi (C-Side)

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",size=[1200,400])#,yAxisRange = [60,180])
    pad1.cd()
    canvas.SetCanvasSize(1000,400)
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,60,(j*2*math.pi)/18,180)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,175,str(j))
    pad1.cd()
    prof = quantileProfile(hist,quantileOrder=0.5)[0]#.ProfileX()
    prof.SetMarkerStyle(21)
    prof.SetMarkerSize(0.4)
    prof.SetMarkerColor(1)
    prof.Draw("SAME P L")
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

#### pT

In [ ]:
%jsroot on
objectName="hPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,log="logxy",drawOption="HIST",size=[800,400])
    canvas.SetCanvasSize(700,400)
    pad1.SetGrid(1)
    canvas.Draw()

#### Q/pT

In [ ]:
%jsroot on
objectName="hQOverPt;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="HIST",size=[800,400])
    canvas.SetCanvasSize(700,400)
    pad1.SetGrid(1)
    canvas.Draw()

### DCA of tracks

#### DCAr vs phi

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

#### DCAr vs eta

In [ ]:
%jsroot on
objectName="hDCArVsEtaPos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[-1,1],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(-1,val,1,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsEtaNeg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[-1,1],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(-1,val,1,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

##### DCAr vs nClusters

#### DCAr vs nClusters

In [ ]:
%jsroot on
objectName="hDCArVsNClsPos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[60,160],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(60,val,160,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsNClsNeg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[60,160],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(60,val,160,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

#### DCAr vs pT

In [ ]:
%jsroot on
objectName="hDCArVsPtPos"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[0.1,20], log="logx",size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(0.1,val,20,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsPtNeg"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[0.1,20], log="logx",size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(0.1,val,20,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

### DCA high pT tracks

#### DCAr vs phi with pT cut (>1 GeV)

In [ ]:
%jsroot on
objectName="hDCAr_A_Pos_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_A_Neg_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Pos_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCAr_C_Neg_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-5,(j*2*math.pi)/18,5)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,4.5,str(j))
    targets = [ROOT.TLine(hist[0].GetXaxis().GetXmin(),val,hist[0].GetXaxis().GetXmax(),val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

#### DCAr vs eta with pT cut (>1 GeV)

In [ ]:
%jsroot on
objectName="hDCArVsEtaPos_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[-1,1],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(-1,val,1,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsEtaNeg_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[-1,1],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(-1,val,1,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

#### DCAr vs nClusters with pT cut (>1 GeV)

In [ ]:
%jsroot on
objectName="hDCArVsNClsPos_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[60,160],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(60,val,160,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

In [ ]:
%jsroot on
objectName="hDCArVsNClsNeg_pTmin"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-5,5], xAxisRange=[60,160],size=[1200,400])
    #[fits,legend2,canvas2] = sliceAndFit(objectName,rootDataFile)
    pad1.cd()
    targets = [ROOT.TLine(60,val,160,val) for val in [-0.2,0,0.2]]
    for i,target in enumerate(targets):
        target.SetLineStyle(1)
        target.SetLineWidth(1)
        if i == 1:
            target.SetLineColor(1)
        else: 
            target.SetLineColor(2)
        target.Draw()
    canvas.SetCanvasSize(1000,400)
    leg = ROOT.TLegend()
    hist[0].FitSlicesY()
    fitMean = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1"))
    fitRMS = copy(ROOT.gDirectory.Get(hist[0].GetName()+"_2"))
    fitMean.SetMarkerStyle(21)
    fitMean.SetMarkerSize(0.4)
    fitMean.SetMarkerColor(1)
    fitMean.Draw("SAME")
    fitRMS.SetMarkerStyle(25)
    fitRMS.SetMarkerSize(0.4)
    fitRMS.SetMarkerColor(2)
    fitRMS.Draw("SAME")
    leg.AddEntry(fitMean,"Mean")
    leg.AddEntry(fitRMS,"RMS")
    leg.Draw()
    canvas.Draw()

## Energy Loss

#### dEdxTot for MIPs

In [ ]:
%jsroot on

# Extract the year from periodName (e.g., "LHC23f" -> "23")
periodYear = periodName[3:5]

# Set fit option based on the year
if periodYear == "23":
    # 2023 data is populated with background tracks between the pion and electrons peaks
    fitOption = "double_gaussian_with_exp"
    print("Fit option: double gaussian with exponential background between pion and electron peaks")
elif periodYear == "24":
    # 2024 data is cleaner and contains a third peak
    fitOption = "triple_gaussian"
    print("Fit option: triple gaussian")
else:
    fitOption = "double_gaussian"
    print("Fit option: double gaussian")

if num_gaussians == 2:
    # Data is populated with background tracks between the pion and electrons peaks
    fitOption = "double_gaussian_and_tail" # double_gaussian, double_gaussian_with_exp, double_gaussian_and_tail
    print("Fit option: double gaussian with exponential background between pion and electron peaks")
elif num_gaussians == 3:
    # Data is cleaner and contains a third peak
    fitOption = "triple_gaussian"
    print("Fit option: triple gaussian")
sepPower, canvas = separationPower(rootDataFile[0], drawFit=True, getProjectionP=True, fit_type=fitOption)
canvas.Draw()



In [ ]:
%jsroot on

objects = ["hdEdxTotMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 48<histogram.GetFunction(\"gaus\").GetParameter(1)<52",printQuality=False)
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,check=quality)
        hists.append(copy(hist[0]))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    leftPad = c.cd(1)
    hists[0].Draw()
    leftPad.SetGrid(1)
    hists[0].SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,hist.GetMinimum(),50,hist.GetMaximum()) for hist in hists]
    # Draw fit stats
    fitMeans = [ROOT.TText(110,hist.GetMaximum()*0.7,"MIP mean = "+f'{hist.GetFunction("gaus").GetParameter(1):.3f}') for hist in hists]
    fitSigmas = [ROOT.TText(110,hist.GetMaximum()*0.62,"MIP sigma = "+f'{hist.GetFunction("gaus").GetParameter(2):.3f}') for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
        fitMeans[i].SetTextAlign(22)
        fitMeans[i].SetTextFont(43)
        fitSigmas[i].SetTextAlign(22)
        fitSigmas[i].SetTextFont(43)
    targets[0].Draw()
    fitMeans[0].Draw()
    fitSigmas[0].Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        hists[i].Draw()
        targets[i].Draw()
        hists[i].SetStats(0)
        fitMeans[i].Draw()
        fitSigmas[i].Draw()
    c.Draw()

#### Moving window MIP dEdxTot

In [ ]:
if checkIfExists(rootDataFile[0],"hdEdxTotMIP_TPC_mw"):
    folder = getHistogram(rootDataFile[0],"hdEdxTotMIP_TPC_mw")
    histTimestamps = list(set([item.GetName() for item in folder.GetListOfKeys()]))
    histTimestamps.sort()
    hists = [folder.Get(time+";1") for time in histTimestamps]
    canvas,leg = drawMovingWindowOverlay(hists,histTimestamps,normalize=True)
    leg.Draw()
    canvas.Draw()

#### dEdxTotMip vs sector (phi)

In [ ]:
%jsroot on

objects = ["hdEdxTotMIPVsSec_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    target.Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        target.Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxTotMip vs nClusters

In [ ]:
%jsroot on
objects = ["hdEdxTotMIPVsNcl_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[60,152],[55,63],[14,34],[12,30],[10,25]]
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxTotMip vs tan(lambda)

In [ ]:
%jsroot on
objects = ["hdEdxTotMIPVsTgl_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[-2,2]]*5
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxTotMip vs sin(phi)

In [ ]:
%jsroot on
objects = ["hdEdxTotMIPVsSnp_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[-.5,.5]]*5
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxTot vs p

In [ ]:
%jsroot on
objects = ["hdEdxTotVsP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz")
        hists.append(copy(hist[0]))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Rebin(2)
    hists[0].Draw("COLZ")
    hists[0].SetStats(0)
    ROOT.gPad.SetLogx(1)
    ROOT.gPad.SetLogy(1)
    ROOT.gPad.SetLogz(1)
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Rebin(2)
        hists[i].Draw("COLZ")
        hists[i].SetStats(0)
        ROOT.gPad.SetLogx(1)
        ROOT.gPad.SetLogy(1)
        ROOT.gPad.SetLogz(1)
    c.Draw()

#### dEdxTot vs p for positive and negative charges

In [ ]:
%jsroot on
objects = ["hdEdxTotVsP_" + region + "_TPC" for region in ["Pos","Neg"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz",normalize=False)
        hists.append(copy(hist[0]))
    c = ROOT.TCanvas("MIP","MIP",1200,400)
    c.Divide(3)
    # Pos
    leftPad = c.cd(1)
    hists[0].Rebin(2)
    hists[0].Draw("COLZ")
    hists[0].SetStats(0)
    ROOT.gPad.SetLogx(1)
    ROOT.gPad.SetLogy(1)
    ROOT.gPad.SetLogz(1)
    # Neg
    middlePad = c.cd(2)
    hists[1].Rebin(2)
    hists[1].Draw("COLZ")
    hists[1].SetStats(0)
    ROOT.gPad.SetLogx(1)
    ROOT.gPad.SetLogy(1)
    ROOT.gPad.SetLogz(1)
    # Ratio
    rightPad = c.cd(3)
    ratioHist = hists[0].Clone("hRatio_dEdxTotvsP")
    ratioHist.Divide(hists[1])
    ratioHist.SetTitle("Ratio: Positive / Negative")
    ratioHist.Draw("COLZ")
    ratioHist.SetStats(0)
    ROOT.gPad.SetLogx(1)
    ROOT.gPad.SetLogy(1)
    ROOT.gPad.SetLogz(1)
    c.Draw()

In [ ]:
%jsroot on
objectName="CdEdxPIDHypothesisVsp"
if checkIfExists(rootDataFile,objectName):
    ROOT.gStyle.SetPalette(55)
    hists = []
    c = getHistogram(rootDataFile[0],objectName)
    for prim in c.GetListOfPrimitives():
        if prim.GetName() in ["CdEdxPIDHypothesisVsp_1", "CdEdxPIDHypothesisVsp_6"]:
            pad = c.GetPrimitive(prim.GetName())
            hist = pad.GetPrimitive(pad.GetListOfPrimitives()[0].GetName())
            hists.append(hist)
            
    canvas = ROOT.TCanvas("","",1000,400)
    canvas.Divide(len(hists))
    for i,hist in enumerate(hists):
        canvas.cd(i+1)
        hist.Draw("COLZ")
        hist.SetStats(0)
        ROOT.gPad.SetLogx(1)
        ROOT.gPad.SetLogy(1)
    canvas.Draw()

In [ ]:
ROOT.gStyle.SetPalette(57)

---

#### dEdxMax for MIPs

In [ ]:
%jsroot on

objects = ["hdEdxMaxMIP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for objectName in objects:
        quality = checkHistograms(objectName,rootDataFile,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",40,60)) and 48<histogram.GetFunction(\"gaus\").GetParameter(1)<52",printQuality=False)
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,check=quality)
        hists.append(copy(hist[0]))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    leftPad = c.cd(1)
    hists[0].Draw()
    leftPad.SetGrid(1)
    hists[0].SetStats(0)
    #Draw target 50
    targets = [ROOT.TLine(50,hist.GetMinimum(),50,hist.GetMaximum()) for hist in hists]
    # Draw fit stats
    fitMeans = [ROOT.TText(110,hist.GetMaximum()*0.7,"MIP mean = "+f'{hist.GetFunction("gaus").GetParameter(1):.3f}') for hist in hists]
    fitSigmas = [ROOT.TText(110,hist.GetMaximum()*0.62,"MIP sigma = "+f'{hist.GetFunction("gaus").GetParameter(2):.3f}') for hist in hists]
    for i in range(len(objects)):
        targets[i].SetLineStyle(2)
        targets[i].SetLineColor(3)
        targets[i].SetLineWidth(3)
        fitMeans[i].SetTextAlign(22)
        fitMeans[i].SetTextFont(43)
        fitSigmas[i].SetTextAlign(22)
        fitSigmas[i].SetTextFont(43)
    targets[0].Draw()
    fitMeans[0].Draw()
    fitSigmas[0].Draw()
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        ROOT.gPad.SetGrid(1)
        hists[i].Draw()
        targets[i].Draw()
        hists[i].SetStats(0)
        fitMeans[i].Draw()
        fitSigmas[i].Draw()
    c.Draw()

#### dEdxMaxMip vs sector (phi)

In [ ]:
%jsroot on
objects = ["hdEdxMaxMIPVsSec_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    for objectName in objects:
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    target = ROOT.TLine(hist[0].GetXaxis().GetXmin(),50,hist[0].GetXaxis().GetXmax(),50)
    target.SetLineStyle(2)
    target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    target.Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        target.Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxMaxMip vs nClusters

In [ ]:
%jsroot on
objects = ["hdEdxMaxMIPVsNcl_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[60,152],[55,63],[14,34],[12,30],[10,25]]
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxMaxMip vs tan(lambda)

In [ ]:
%jsroot on
objects = ["hdEdxMaxMIPVsTgl_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[-2,2]]*5
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxMaxMip vs sin(phi)

In [ ]:
%jsroot on
objects = ["hdEdxMaxMIPVsSnp_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    fits = []
    ranges = [[-.5,.5]]*5
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",xAxisRange=ranges[i],yAxisRange=[30,70])
        hist[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
        hists.append(copy(hist[0]))
        fits.append(copy(ROOT.gDirectory.Get(hist[0].GetName()+"_1")))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    targets = [ROOT.TLine(range[0],50,range[1],50) for range in ranges]
    for target in targets:
        target.SetLineStyle(2)
        target.SetLineWidth(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Draw("COLZ")
    targets[0].Draw()
    hists[0].SetStats(0)
    hists[0].FitSlicesY(ROOT.TF1("myFunc","gaus",40,60))
    fits[0].SetMarkerStyle(21)
    fits[0].SetMarkerSize(0.4)
    fits[0].SetMarkerColor(1)
    fits[0].Draw("SAME")
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Draw("COLZ")
        targets[i].Draw()
        hists[i].SetStats(0)
        fits[i].SetMarkerStyle(21)
        fits[i].SetMarkerSize(0.4)
        fits[i].SetMarkerColor(1)
        fits[i].Draw("SAME")
    c.Draw()

#### dEdxMax vs p

In [ ]:
%jsroot on
objects = ["hdEdxMaxVsP_" + region for region in ["TPC","IROC","OROC1","OROC2","OROC3"]]
if all([checkIfExists(rootDataFile,objectName) for objectName in objects]):
    hists = []
    for i,objectName in enumerate(objects):
        [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",log="logxyz")
        hists.append(copy(hist[0]))
    c = ROOT.TCanvas("MIP","MIP",1200,600)
    c.Divide(2)
    # TPC
    leftPad = c.cd(1)
    hists[0].Rebin(2)
    hists[0].Draw("COLZ")
    ROOT.gPad.SetLogx(1)
    ROOT.gPad.SetLogy(1)
    ROOT.gPad.SetLogz(1)
    hists[0].SetStats(0)
    # ROCs
    rightPad = c.cd(2)
    rightPad.Divide(2,2)
    for i in range(1,5):
        rightPad.cd(i)
        hists[i].Rebin(2)
        hists[i].Draw("COLZ")
        hists[i].SetStats(0)
        ROOT.gPad.SetLogx(1)
        ROOT.gPad.SetLogy(1)
        ROOT.gPad.SetLogz(1)
    c.Draw()